This code was run multiple times with manual modifications, to gather weather data for London-based teams. I downloaded  It will no longer run, as the API key has been reset.

In [16]:
import pandas as pd
import requests
import datetime
import json
import numpy as np
from os.path import isfile
import time

In [2]:
with open('PLmatches.csv', 'r') as open_file:
    df = pd.read_csv(open_file, index_col=0)

In [1]:
API_key = '1c0e0cc4529f6cafd3ab0ef69720cb2a'

In [55]:
def get_row_weather(row):
    #time.sleep(0.01)
    completed_teams = ['Arsenal', 'Tottenham Hotspur', 'West Ham United', 'Crystal Palace', 'Fulham', 'Charlton Athletic', 'Chelsea', 'Queens Park Rangers']
    if row['hometeam_name'] in completed_teams:
        return pd.Series({"precip_prob": row['precip_prob'], "temp_high": row['temp_high']})
    if row['hometeam_name'] != 'Watford':
        return pd.Series({"precip_prob": np.nan, "temp_high": np.nan})
    match_time = datetime.datetime.utcfromtimestamp(row['match_timestamp']//1000)
    match_midnight = match_time.replace(hour=0, minute=0, second=0)
    match_midnight = match_midnight + datetime.timedelta(days=1) #oops, the data is from the previous day
    mdnt_ts = int(match_midnight.timestamp())
    Ndec = str(row['Ndec'])
    Wdec = str(row['Wdec'])

    if isfile(f'london_weatherdata/{mdnt_ts}.json'):
        print(f'opening {mdnt_ts} from file')
        with open(f'london_weatherdata/{mdnt_ts}.json') as open_file:
            weather = json.load(open_file)
    else:
        print(f'querying for {mdnt_ts}')
        test = requests.get(f'https://api.darksky.net/forecast/{API_key}/{Ndec},{Wdec},{mdnt_ts}')
        weather = test.json()

        with open(f'london_weatherdata/{mdnt_ts}.json', 'w') as open_file:
            json.dump(weather, open_file)
    try:
        precip_prob = weather['daily']['data'][0]['precipProbability']
    except KeyError:
        precip_prob = np.nan
    try:
        temp_high = weather['daily']['data'][0]['temperatureHigh']
    except KeyError:
        temp_high = np.nan
        
    return pd.Series({"precip_prob": precip_prob, "temp_high": temp_high})

In [59]:
df[['precip_prob', 'temp_high']] = df.apply(get_row_weather, axis=1)

opening 1156316400 from file
opening 1156662000 from file
opening 1158476400 from file
opening 1159858800 from file
opening 1162105200 from file
opening 1162713600 from file
opening 1164787200 from file
opening 1165737600 from file
opening 1167206400 from file
opening 1168761600 from file
opening 1169625600 from file
opening 1170576000 from file
opening 1172131200 from file
opening 1172390400 from file
opening 1172995200 from file
opening 1175410800 from file
opening 1176188400 from file
opening 1177225200 from file
opening 1179126000 from file
opening 1439708400 from file
opening 1440399600 from file
opening 1442127600 from file
opening 1443423600 from file
opening 1445151600 from file
opening 1446361200 from file
opening 1448179200 from file
opening 1449388800 from file
opening 1450684800 from file
opening 1451376000 from file
opening 1451808000 from file
opening 1453622400 from file
querying for 1454572800
opening 1456646400 from file
opening 1457251200 from file
opening 1458457200 

In [49]:
df[df['hometeam_name'] == 'Crystal Palace']

,hometeam_name,awayteam_name,home_goals,away_goals,match_date,match_timestamp,match_referee,match_stadium,match_attendance,home_possession,...,away_Ndec,away_Wdec,away_distance,precip_prob,temp_high,week_power,home_last3_leaguepts,away_last3_leaguepts,home_leaguepts_gained,away_leaguepts_gained
9239,Crystal Palace,Tottenham Hotspur,0,1,Sun 18 Aug 2013,1376829000000,Mark Clattenburg,"Selhurst Park, London",23285.0,41.8,...,51.5558,-0.2797,22.111220,NaN,NaN,10,NaN,2.333333,0,3
9253,Crystal Palace,Sunderland,3,1,Sat 31 Aug 2013,1377966600000,Lee Probert,"Selhurst Park, London",22671.0,50.0,...,54.9144,-1.3883,400.870512,NaN,NaN,8,0.000000,0.333333,3,0
9279,Crystal Palace,Swansea City,0,2,Sun 22 Sep 2013,1379853000000,Kevin Friend,"Selhurst Park, London",22466.0,43.4,...,51.6422,-3.9351,268.535404,NaN,NaN,6,1.000000,1.333333,0,3
9310,Crystal Palace,Fulham,1,4,Mon 21 Oct 2013,1382382000000,Lee Mason,"Selhurst Park, London",24881.0,45.9,...,51.4750,-0.2217,12.743033,NaN,NaN,3,0.000000,1.000000,0,3
9312,Crystal Palace,Arsenal,0,2,Sat 26 Oct 2013,1382787900000,Chris Foy,"Selhurst Park, London",20050.0,37.5,...,51.5550,-0.1086,17.507111,NaN,NaN,2,0.000000,2.333333,0,3
9333,Crystal Palace,Everton,0,0,Sat 9 Nov 2013,1384009200000,Craig Pawson,"Selhurst Park, London",25231.0,27.2,...,53.4389,-2.9664,299.908406,NaN,NaN,0,0.000000,2.333333,1,1
9361,Crystal Palace,West Ham United,1,0,Tue 3 Dec 2013,1386100800000,Lee Mason,"Selhurst Park, London",23891.0,37.0,...,51.5386,-0.0164,16.333267,NaN,NaN,3,1.333333,1.000000,3,0
9371,Crystal Palace,Cardiff City,2,0,Sat 7 Dec 2013,1386428400000,Mike Jones,"Selhurst Park, London",23705.0,43.7,...,51.4728,-3.2031,216.927981,NaN,NaN,4,2.000000,0.666667,3,0
9391,Crystal Palace,Newcastle United,0,3,Sat 21 Dec 2013,1387638000000,Neil Swarbrick,"Selhurst Park, London",24936.0,33.1,...,54.9756,-1.6217,411.118602,NaN,NaN,6,2.000000,1.333333,0,3
9422,Crystal Palace,Norwich City,1,1,Wed 1 Jan 2014,1388588400000,Mike Dean,"Selhurst Park, London",25189.0,45.1,...,52.6221,1.3087,166.448454,NaN,NaN,9,1.000000,0.333333,1,1


In [180]:
df.sample(10)

,hometeam_name,awayteam_name,home_goals,away_goals,match_date,match_timestamp,match_referee,match_stadium,match_attendance,home_possession,...,away_last3_goals,home_last3_poss,away_last3_poss,Ndec,Wdec,away_Ndec,away_Wdec,away_distance,precip_prob,temp_high
6300,Sunderland,Middlesbrough,3,2,Sat 26 Apr 2008,1209218400000,Steve Bennett,"Stadium of Light, Sunderland",NaN,60.1,...,1.000000,45.100000,48.666667,54.9144,-1.3883,54.5783,-1.2169,39.008398,NaN,NaN
9850,West Ham United,Manchester United,1,1,Sun 8 Feb 2015,1423412100000,Mark Clattenburg,"Boleyn Ground, London",34499.0,41.2,...,1.666667,42.400000,64.533333,51.5319,0.0394,53.4631,-2.2914,266.873365,NaN,NaN
6199,Everton,Reading,1,0,Sat 9 Feb 2008,1202569200000,Mark Halsey,"Goodison Park, Liverpool",NaN,50.2,...,0.000000,45.366667,45.866667,53.4389,-2.9664,51.4222,-0.9828,261.820676,NaN,NaN
5867,Charlton Athletic,Wigan Athletic,1,0,Sat 31 Mar 2007,1175349600000,Peter Walton,"The Valley, London",26500.0,54.2,...,0.666667,44.700000,47.533333,51.4864,0.0364,53.5478,-2.6539,293.164434,NaN,NaN
9611,Arsenal,Crystal Palace,2,1,Sat 16 Aug 2014,1408206600000,Jonathan Moss,"Emirates Stadium, London",59962.0,76.9,...,1.666667,61.366667,38.400000,51.5550,-0.1086,51.3983,-0.0856,17.507111,0.64,67.75
14194,West Bromwich Albion,Swansea City,3,1,Wed 14 Dec 2016,1481745600000,Michael Oliver,"The Hawthorns, West Bromwich",21467.0,39.5,...,2.666667,40.033333,52.733333,52.5092,-1.9639,51.6422,-3.9351,166.038238,NaN,NaN
9564,Swansea City,Chelsea,0,1,Sun 13 Apr 2014,1397401620000,Phil Dowd,"Liberty Stadium, Swansea",20761.0,38.2,...,3.000000,51.700000,57.033333,51.6422,-3.9351,51.4817,-0.1911,260.217263,NaN,NaN
8231,Sunderland,Southampton,1,1,Sun 12 May 2013,1368367200000,Mike Dean,"Stadium of Light, Sunderland",41988.0,36.0,...,0.000000,42.833333,52.500000,54.9144,-1.3883,50.9058,-1.3911,446.094906,NaN,NaN
6744,Liverpool,Burnley,4,0,Sat 12 Sep 2009,1252764000000,Lee Mason,"Anfield, Liverpool",43817.0,60.4,...,0.666667,73.700000,37.966667,53.4308,-2.9608,53.7892,-2.2303,62.679871,NaN,NaN
14290,Chelsea,Swansea City,3,1,Sat 25 Feb 2017,1488034800000,Neil Swarbrick,"Stamford Bridge, London",41612.0,65.6,...,1.666667,50.333333,41.100000,51.4817,-0.1911,51.6422,-3.9351,260.217263,NaN,NaN


In [60]:
with open('PLmatches.csv', 'w') as open_file:
    df.to_csv(open_file)